In [4]:
import sys
sys.path.append('../')
import Phase_Detector.Utility as Utility
import torch

In [5]:
from torch.nn import CrossEntropyLoss
import torch
loss = CrossEntropyLoss()
print(loss)

CrossEntropyLoss()


In [48]:
y_true = [1,0,0,1,0,0,1,0,0,]
y_pre_tensor = [torch.tensor([0.1,0.8]).reshape(1,2) for i in range(len(y_true))]
metric = Utility.Metrics(2,y_pred=y_pre_tensor, y_true_label=y_true)
#metric.get_accuracy()

Test Dataloader

In [15]:
import torch
from torch.utils.data import Dataset, DataLoader

# 定义一个简单的自定义数据集
# 示例数据
data = [i for i in range(10)]  # 一个包含0-9的简单列表

# 创建Dataset
simple_dataset = ImageDataset(data)

# 创建DataLoader
#data_loader = DataLoader(simple_dataset, batch_size=2, shuffle=True)

# 迭代DataLoader
#for batch in data_loader:
 #   print(batch)


c:\Users\098986\AppData\Local\anaconda3\envs\CILM\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from torch import tensor
import torch
import nibabel as nib

from torch.utils.data import WeightedRandomSampler

import numpy as np
import pandas as pd
import os
from monai.transforms import Compose

from monai.data import ImageDataset,DataLoader


class Image_Dataset(ImageDataset):
    def __init__(self,image_files,labels,transform_methods=None,data_aug=True,label_name=None,*args,**kwargs):
        """
        args:
            image_files: list of image files path
            labels: list of labels
            transform_methods: list of transform methods
            data_aug: True if data augmentation is used
            label_name: name of the label
        
        """
        if data_aug:
            transform = Compose(transform_methods)
        else:
            transform = None

        super().__init__(image_files=image_files,labels=labels,transform=transform,*args, **kwargs)

    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self,index):
        image = self.image_files[index]
        label = self.labels[index]

        # get image array
        image = nib.load(image).get_fdata()
        # here to do windowing

        image = tensor(image)
        return image,label
    
class Data_Loader(DataLoader):
    def __init__(self,dataset,batch_size,num_workers=0,*args,**kwargs):
        super().__init__(dataset=dataset,batch_size=batch_size,num_workers=num_workers,*args,**kwargs)
    
    
    def build_train_loader(self):
        return DataLoader(self.dataset,batch_size=self.batch_size,shuffle=True,num_workers=self.num_workers,drop_last=True,*self.args,**self.kwargs)

    def build_vali_loader(self):
        return DataLoader(self.dataset,batch_size=self.batch_size,shuffle=False,num_workers=self.num_workers,drop_last=False,*self.args,**self.kwargs)
    
    def build_test_loader(self):
        return DataLoader(self.dataset,batch_size=self.batch_size,shuffle=False,num_workers=self.num_workers,drop_last=False,*self.args,**self.kwargs)


In [11]:

import monai
import torch

model = monai.networks.nets.resnet10()
model.parameters()

<generator object Module.parameters at 0x00000186E6371F20>

In [33]:
def build_optimizer(parameters,**kwargs):
    print(kwargs)
    return torch.optim.Adam(parameters,**kwargs)


optimizer_param = {"lr":0.001}
optimizer = build_optimizer(model.parameters(),**optimizer_param)

{'lr': 0.001}


In [46]:
from torch.nn import CrossEntropyLoss

class Loss:
    def __init__(self,*args,**kwargs):
        """
        args:
            args only have one loss function
        """
        self.args = args

    def build_loss(self):
        return self.args[0]
    
    def calculate_loss(self,*args):
        pass


CrossEntropyLoss()

In [20]:
from monai.transforms import Compose,EnsureChannelFirst,RandZoom,RandRotate,RandFlip,ToTensor
import inspect
transform_param = {"transform_methods":[
                                EnsureChannelFirst(),
                                # Data augmentation
                                RandZoom(prob = 0.5, min_zoom=1.0, max_zoom=1.2),
                                RandRotate(range_z = 0.35, prob = 0.8),
                                RandFlip(prob = 0.5),
                                # To tensor
                                ToTensor()
                                ]}
transform_param['transform_methods'][1].__dict__


{'_do_transform': True,
 'prob': 0.5,
 '_lazy': False,
 'min_zoom': (1.0,),
 'max_zoom': (1.2,),
 'mode': area,
 'padding_mode': edge,
 'align_corners': None,
 'dtype': torch.float32,
 'keep_size': True,
 'kwargs': {},
 '_zoom': [1.0]}

Test Dataloader


In [8]:
from sklearn.metrics import accuracy_score
import torch
a = torch.tensor([0,1])
b = [0,]
accuracy_score(a,b)

0.5

In [14]:
a = Data_Loader(dataset=dataset,batch_size=1,num_workers=0).build_train_loader()
len(a)

40

# Binary classifaction


In [2]:
import pandas as pd
pvp_label = pd.read_csv('../../Data/CT_Phase/True_Label/Phase_PVP.csv')
#将Phase中的Phase的1和0都换成0,2换成1
pvp_label['Phase'] = pvp_label['Phase'].apply(lambda x: 0 if x == 1 or x==0 else 1)


In [6]:
pvp_label.to_csv('../../Data/CT_Phase/True_Label/Phase_PVP.csv',index=False)


